
Переиспользуем код с предыдущего шага для проверки своей реализации сверточного слоя.

Рассмотрим свертку батча из одного однослойного изображения $3*3$ с ядром из одного фильтра $2*2$, stride = 1, то есть, на выходе должна получиться одна матрица $2*2$. Строго записанная размерность выхода равна (1 - изображений в батче, 1 - количество фильтров в ядре, 2 - высота матрицы выхода, 2 - ширина матрицы выхода).
  
  
Пусть W - веса ядра, X - вход, Y - выход.
<img src="../data/untitled (11).png" width="500" height="600" >


Вычислить выход можно в цикле:

<img src="../data/untitled (12).png" width="500">

 

На каждой итерации цикла фильтр умножается попиксельно на часть изображения, а потом 4 получившиеся числа складываются - получается один пиксель выхода.

Требуемое количество итераций для данного случая - 4, так как может быть 2 положения ядра и 2 по вертикали, общее число итераций - произведение количеств положений, то есть в данном случае 2*2 = 4.

Давайте перейдем от простого случая к общему.

- Если бы изображение было многослойным, например трехслойное - RGB, значит, фильтры в ядре тоже должны быть трехслойные. Каждый слой фильтра попиксельно умножается на соответствующий слой исходного изображения. То есть в данном случае после умножения получилось бы 4*3 = 12 произведений, результаты которых складываются, и получается значение выходного пикселя.
  
 
- Если бы фильтров в ядре было больше одного, то добавился бы внешний цикл по фильтрам, внутри которого мы считаем свертку для каждого фильтра.


- Если бы во входном батче было более 1 изображения, то добавился бы еще один внешний цикл по изображениям в батче.

**Напоминание: во всех шагах этого урока мы считаем bias в сверточных слоях нулевым.**

На этом шаге требуется реализовать сверточный слой через циклы.

Обратите внимание, что в коде рассматривается общий случай - батч на входе не обязательно состоит из одного изображения, в ядре несколько слоев.

In [28]:
import torch
from abc import ABC, abstractmethod


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    return batch_size, out_channels, output_height, output_width

def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [
                          [
                              [[0., 1, 0],
                               [1,  2, 1],
                               [0,  1, 0]],
                              
                              [[1, 2, 1],
                               [0, 3, 3],
                               [0, 1, 10]],
                              
                              [[10, 11, 12],
                               [13, 14, 15],
                               [16, 17, 18]]
                          ]
                      ]
    )

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(
        0,
        batch_size * in_channels * input_height * input_width,
        out=torch.FloatTensor()
    ).reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(Conv2d, stride, kernel, input_tensor)

    
    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    """
        self.in_channels: int
        self.out_channels: int
        self.kernel_size: int
        self.stride: int
        
        self.kernel: torch.tensor
    """
    def __call__(self, batch):
        shape = calc_out_shape(batch.shape, self.out_channels, self.kernel_size, self.stride, padding=0)
        output = torch.zeros(shape)
        # print("shape:", shape)
        pics, filts, rows, cols = shape
        for i in range(pics):
            for fil in range(filts):
                for r in range(rows):
                    r = r * self.stride
                    for c in range(cols):
                        c = c * self.stride
                        output[i, fil, r, c] = (batch[i, :, r:r+self.kernel_size, c:c+self.kernel_size] * 
                                                self.kernel[fil, :, r:r+self.kernel_size, c:c+self.kernel_size]).sum()
        # print(output)
        return output

# Корректность реализации определится в сравнии со стандартным слоем из pytorch.
# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# print(test_conv2d_layer(Conv2dLoop))